<a href="https://colab.research.google.com/github/mishalsherif/Gen-AI-text-generator/blob/main/GEN_AI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install transformers

In [13]:
from transformers import pipeline

In [14]:
generator = pipeline('text-generation', model='gpt2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
prompt = "you are college teacher. write a story for 10-year old : \nOnce upon a time, a little boy "
output = generator(prompt, max_length=50)

print(output[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


you are college teacher. write a story for 10-year old : 
Once upon a time, a little boy  was living in a place that had just started to become a place.  He was not only good at math, but he was better at writing.  He had a great sense of humor and he was very social.  The story was about him being pulled out of the house by a man who had a huge gun and told to give him a gun.  The whole story is about his teacher being taken away from him and being shot by another teacher.  There is a story about the teacher being shot so that the teacher can get away from the guy and leave.  The teacher gets hit by the gun and he is killed by the shooter.  It was a very important story and I think that the students would have said something similar about it on their own.
I think that the story about the teacher being killed by another teacher is that of a person who is already dead.  I think that it does not make sense for the students to be scared that their teacher is going to die.  I think that th

In [16]:
!pip install -q transformers gradio accelerate

In [17]:
import gradio as gr
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [18]:
device = "cuda" if torch.cuda.is_available()else "cpu"
MODEL_NAME = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

In [19]:
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [26]:
def generate_text(prompt, max_new_tokens=80, temperature=0.9):
  if not prompt.strip():
    return "Please enter a prompt"

  inputs = tokenizer(prompt, return_tensors="pt").to(device)

  with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [27]:
import torch
from transformers import GPT2Tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [37]:
with gr.Blocks() as demo:
  gr.Markdown("# GPT-2 Text Generation")
  gr.Markdown("Fast, lightweight, and gpu-aware")

  prompt = gr.Textbox(label="Enter a prompt",
  placeholder  = "Once upon a time in a world",
  lines = 3
  )

  with gr.Row():
    max_new_tokens = gr.Slider(20, 200, value = 80, label = "Max New Tokens")
    temperature = gr.Slider(0.3, 1.3, value = 0.9, label = "Temperature")

  generate_button = gr.Button("Generate")

  output = gr.Textbox(label="Generated Text", lines=10)
  generate_button.click(
      generate_text,
      inputs = [prompt, max_new_tokens, temperature],
      outputs = output
  )

In [38]:
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bcef50d6d79bbf8ba7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
